In [ ]:
import os
import cv2
import numpy as np
import dlib
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import joblib
import time

# Load Dlib facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Paths to datasets
down_syndrome_path = 'downSyndrome'
healthy_path = 'healty'

# Constants
RADIUS = 1
POINTS = 8 * RADIUS
METHOD = 'uniform'
PATCH_SIZE = 32

landmark_indices = [36, 39, 42, 45, 27, 30, 33, 31, 35, 51, 48, 54, 57, 68]

pairs = [
    (36, 39), (39, 42), (42, 45), (27, 30), (30, 33), (33, 31),
    (33, 35), (30, 31), (30, 35), (33, 51), (51, 48), (51, 54),
    (51, 57), (48, 57), (54, 57), (39, 68), (42, 68)
]

2024-12-04 15:58:36.089057: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 15:58:36.092212: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 15:58:36.166553: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 15:58:36.206286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733302716.302974   13428 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733302716.34

In [ ]:
def is_frontal_face(landmarks):
    left_eye = np.mean(np.array(landmarks[36:42]), axis=0)
    right_eye = np.mean(np.array(landmarks[42:48]), axis=0)
    nose_tip = np.array(landmarks[30])
    eye_distance = np.linalg.norm(left_eye - right_eye)
    nose_to_left_eye = np.linalg.norm(nose_tip - left_eye)
    nose_to_right_eye = np.linalg.norm(nose_tip - right_eye)
    symmetry_threshold = 0.3 * eye_distance
    return abs(nose_to_left_eye - nose_to_right_eye) < symmetry_threshold

def get_landmarks(image_input):
    gray = cv2.cvtColor(image_input, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        midpoint_x = (points[21][0] + points[22][0]) // 2
        midpoint_y = (points[21][1] + points[22][1]) // 2
        points.append((midpoint_x, midpoint_y))
        # if is_frontal_face(points):
        return points
    return None

def crop_face(image, landmarks, padding_percentage=0.1):
    min_x = min(landmarks, key=lambda x: x[0])[0]
    max_x = max(landmarks, key=lambda x: x[0])[0]
    min_y = min(landmarks, key=lambda x: x[1])[1]
    max_y = max(landmarks, key=lambda x: x[1])[1]

    width = max_x - min_x
    height = max_y - min_y
    max_dimension = max(width, height)

    padding = int(max_dimension * padding_percentage)
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2

    available_left = min_x
    available_right = image.shape[1] - max_x
    available_top = min_y
    available_bottom = image.shape[0] - max_y

    if available_left >= padding and available_right >= padding and available_top >= padding and available_bottom >= padding:
        actual_padding = padding
    else:
        actual_padding = min(available_left, available_right, available_top, available_bottom)

    size = max_dimension + actual_padding * 2

    new_min_x = max(int(center_x - size // 2), 0)
    new_max_x = min(int(center_x + size // 2), image.shape[1])
    new_min_y = max(int(center_y - size // 2), 0)
    new_max_y = min(int(center_y + size // 2), image.shape[0])

    cropped_image = image[new_min_y:new_max_y, new_min_x:new_max_x]
    cropped_image_resized = cv2.resize(cropped_image, (300, 300))

    return cropped_image_resized

def augment_image(image):
    augmented_images = []

    # Flip horizontally
    augmented_images.append(cv2.flip(image, 1))
    
    rows, cols = image.shape[:2]

    # Rotate images
    for angle in [-15, -10, 0, 10, 15]:
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (cols, rows))
        augmented_images.append(rotated)

    # Add Gaussian noise
    noise = np.random.normal(0, 10, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    augmented_images.append(noisy_image)

    # Adjust brightness and contrast
    for alpha in [0.8, 1.2]:  # Brightness scaling
        brightened = cv2.convertScaleAbs(image, alpha=alpha, beta=0)
        augmented_images.append(brightened)

    # Sharpen the image
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    augmented_images.append(sharpened)

    return augmented_images

def extract_patches(image, landmarks, indices, patch_size=PATCH_SIZE):
    patches = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for idx in indices:
        (x, y) = landmarks[idx]
        x_start = max(x - patch_size // 2, 0)
        y_start = max(y - patch_size // 2, 0)
        x_end = min(x + patch_size // 2, gray.shape[1])
        y_end = min(y + patch_size // 2, gray.shape[0])
        patch = gray[y_start:y_end, x_start:x_end]
        if patch.size > 0:
            patches.append(patch)
    return patches

def extract_lbp_from_patches(patches, radius=RADIUS, points=POINTS, method=METHOD):
    lbp_features = []
    for patch in patches:
        lbp = local_binary_pattern(patch, points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.extend(hist)
    return lbp_features

def extract_geometric_features(landmarks, pairs):
    geom_features = []
    for i, j in pairs:
        p1 = landmarks[i]
        p2 = landmarks[j]
        distance = np.linalg.norm(np.array(p1) - np.array(p2))
        geom_features.append(distance)
    return geom_features

def get_combined_features(image, pairs):
    landmarks = get_landmarks(image)
    if landmarks:
        cropped_image = crop_face(image, landmarks)
        cropped_landmarks = get_landmarks(cropped_image)
        if cropped_landmarks:
            patches = extract_patches(cropped_image, cropped_landmarks, landmark_indices)
            lbp_features = extract_lbp_from_patches(patches)
            geom_features = extract_geometric_features(cropped_landmarks, pairs)
            return lbp_features + geom_features
    return None


In [3]:
# Collect data
X = []
y = []
fixed_feature_size = None  # Initialize the fixed size

for path, label in [(down_syndrome_path, 1), (healthy_path, 0)]:
    for img_file in os.listdir(path):
        img_path = os.path.join(path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            combined_features = get_combined_features(img, pairs)
            if combined_features:
                # Initialize fixed size based on the first feature
                if fixed_feature_size is None:
                    fixed_feature_size = len(combined_features)

                # Add features to the dataset
                X.append(combined_features)
                y.append(label)

                # Apply data augmentation
                augmented_images = augment_image(img)
                for aug_img in augmented_images:
                    aug_features = get_combined_features(aug_img, pairs)
                    if aug_features:
                        X.append(aug_features)
                        y.append(label)

# Ensure consistency in feature sizes
def pad_or_truncate(features, fixed_size):
    if len(features) > fixed_size:
        return features[:fixed_size]  # Truncate
    else:
        return np.pad(features, (0, fixed_size - len(features)), mode='constant')  # Pad

X = [pad_or_truncate(features, fixed_feature_size) for features in X]  # Fix feature sizes
X = np.array(X)  # Convert to a NumPy array
y = np.array(y)  # Convert labels to a NumPy array

# Handle NaNs and scale features
X = np.nan_to_num(X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dimensionality reduction (optional)
# pca = PCA(n_components=70)
# X = pca.fit_transform(X)


In [4]:
# # Dimensionality reduction (optional)
# pca = PCA(n_components=80)
# X = pca.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
# Initialize the model with the best parameters found
model = SVC(C=0.5, class_weight='balanced', gamma='auto', kernel='rbf', probability=True)

# Train the model with the training data
start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time
print(f"Training Time: {training_time:.2f} seconds")


Training Time: 65.20 seconds


In [6]:
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
print(f"5-Fold Cross-Validation Accuracy: {np.mean(cv_scores):.2f}")

# Evaluate on the test set
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test Precision: {precision_test:.2f}")
print(f"Test Recall: {recall_test:.2f}")
print(f"Test AUC: {auc_test:.2f}")
print("Test Classification Report:")
print(classification_report(y_test, y_pred_test))

# Evaluate on the training set
y_pred_train = model.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

print(f"Training Accuracy: {accuracy_train:.2f}")
print(f"Training Precision: {precision_train:.2f}")
print(f"Training Recall: {recall_train:.2f}")
print(f"Training AUC: {auc_train:.2f}")
print("Training Classification Report:")
print(classification_report(y_train, y_pred_train))

5-Fold Cross-Validation Accuracy: 0.89
Test Accuracy: 0.89
Test Precision: 0.88
Test Recall: 0.90
Test AUC: 0.96
Test Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      2484
           1       0.88      0.90      0.89      2224

    accuracy                           0.89      4708
   macro avg       0.89      0.89      0.89      4708
weighted avg       0.89      0.89      0.89      4708

Training Accuracy: 0.92
Training Precision: 0.91
Training Recall: 0.93
Training AUC: 0.97
Training Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      5758
           1       0.91      0.93      0.92      5226

    accuracy                           0.92     10984
   macro avg       0.92      0.92      0.92     10984
weighted avg       0.92      0.92      0.92     10984



In [7]:
import pickle


# Combine scaler, PCA, and model into a single dictionary
model_pipeline = {
    'scaler': scaler,
    'model': model,
    # 'pca' : pca
}

# Save to a .pkl file
with open('trained_model_v5.pkl', 'wb') as file:
    pickle.dump(model_pipeline, file)

print("Model saved successfully to trained_model.pkl")



Model saved successfully to trained_model.pkl
